# Fractional Gates

This page introduces two newly supported gate types on the IBM Quantum fleet of QPUs, $R_{ZZ}(\theta)$ and $R_X(\theta)$, and discusses the usecases which enable your quantum workflows to become more efficient. 

### Page outline

- Go over what these gates are
- Explain the situation in which these gates can be used (and why they are more efficient).
  - Go over the example usecase of simulating an Ising system.
- Explain the syntax for these gates.
- How do they differ from parametrized gates.   

## Where to use fractional gates

When constructing circuits containing single or two-qubit rotations and transpiling to an IBM Quantum backend, the circuit is restricted to only execute 

In [4]:
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np

qc = QuantumCircuit(1)
qc.rx(0.5*np.pi, 0)
qc.draw()

┌─────────┐
q: ┤ Rx(π/2) ├
   └─────────┘

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy()